In [2]:
import pandas as pd
import numpy as np

# from sklearn.preprocessing import Imputer
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier, plot_importance
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'xgboost'

# 전처리

In [2]:
train_ = pd.read_excel('C:/Users/user/Desktop/PJT/Statistical_Data_Idea_Contest/DATA/데이터/기술통계19.xlsx',index_col='global_id')
service_ = pd.read_excel('C:/Users/user/Desktop/PJT/Statistical_Data_Idea_Contest/DATA/데이터/실태조사_서비스업19.xlsx',index_col='global_id')
normal_ = pd.read_excel('C:/Users/user/Desktop/PJT/Statistical_Data_Idea_Contest/DATA/데이터/실태조사_일반항목19.xlsx',index_col='global_id')
product_ = pd.read_excel('C:/Users/user/Desktop/PJT/Statistical_Data_Idea_Contest/DATA/데이터/실태조사_제조업19.xlsx',index_col='global_id')

In [205]:
train = train_[:]
service = service_[:]

normal = normal_[:]
product = product_[:]

In [206]:
#여기는 시험용임
service = product_[:]

손익계산서 변수들만 추출

In [207]:
cols = [c for c in service.columns if c[:4] == 'INC_']
service=service[cols]

'-', 콤마 제거

In [208]:
for i in cols:
    service[i] = service[i].replace(' - ',np.nan)
    service[i] = service[i].replace(',','')    

object 타입 float으로 변경

In [209]:
for i in cols:
    service = service.astype({i: float})

2018년 INC_A_21 을 제외한 1~34 제거

In [210]:
cols2 = ['INC_A_01','INC_A_02','INC_A_03','INC_A_04','INC_A_05','INC_A_06','INC_A_07','INC_A_08','INC_A_09','INC_A_10','INC_A_11','INC_A_12','INC_A_13','INC_A_14','INC_A_15','INC_A_16','INC_A_17','INC_A_18','INC_A_19','INC_A_20','INC_A_22','INC_A_23','INC_A_24','INC_A_25','INC_A_26','INC_A_27','INC_A_28','INC_A_29','INC_A_30','INC_A_31','INC_A_32','INC_A_33','INC_A_34']
service.drop(service[cols2],axis='columns',inplace=True)

y 변수 생성을 위한 결측치 제거

In [211]:
service.INC_A_21 = service.INC_A_21.fillna(service.INC_A_21.mean())

y변수 생성
즉 성장률

In [212]:
service['y']=(service['INC_A_21']-service['INC_A_61'])/service['INC_A_61']

손익계산서 변수 中 독립적인 변수만 추출

제거된 변수
*영업손익과 관련된 변수 매출 총손익(41+42) -  판매비와 관리비(45~60까지의 합)
*영업외 수익 : 이자수익 + 외환차익 및 외화환산이익 + 기타 영업외 수익 (63+64+65)
*영업외 비용 : 이자비용 + 외환차손 및 외화환산손실 + 기타 영업외 비용 (67+68+69)
*법인세 비용차감 전 계속사업손익 (61+62-66)
*계속사업손익 (70-71)
*당기순손익 (72+73)
*17년 영업 손익

In [213]:
cols3= ['INC_A_41','INC_A_42','INC_A_45','INC_A_46','INC_A_47','INC_A_48','INC_A_49','INC_A_50','INC_A_51','INC_A_52','INC_A_53','INC_A_54','INC_A_55','INC_A_56','INC_A_57','INC_A_58','INC_A_59','INC_A_60','INC_A_61','INC_A_63','INC_A_64','INC_A_65','INC_A_67','INC_A_68','INC_A_69','INC_A_71','INC_A_73','y']
service = service[cols3]

결측치 처리

In [214]:
service.isnull().sum()

INC_A_41    0
INC_A_42    0
INC_A_45    0
INC_A_46    0
INC_A_47    0
INC_A_48    0
INC_A_49    0
INC_A_50    0
INC_A_51    0
INC_A_52    0
INC_A_53    0
INC_A_54    0
INC_A_55    0
INC_A_56    0
INC_A_57    0
INC_A_58    0
INC_A_59    0
INC_A_60    0
INC_A_61    0
INC_A_63    0
INC_A_64    0
INC_A_65    0
INC_A_67    0
INC_A_68    0
INC_A_69    0
INC_A_71    0
INC_A_73    0
y           0
dtype: int64

8500개의 데이터 中 결측치 비율이 높은 변수 제거

In [215]:
delete_na = ['INC_A_53','INC_A_54', 'INC_A_58', 'INC_A_59', 'INC_A_64', 'INC_A_68', 'INC_A_73']
service.drop(service[delete_na],axis='columns',inplace=True)

결측치 대체 *일단 평균으로 대체

In [216]:
for i in service.columns:
    service[i] = service[i].fillna(service[i].mean())

In [217]:
service.head(8)

,INC_A_41,INC_A_42,INC_A_45,INC_A_46,INC_A_47,INC_A_48,INC_A_49,INC_A_50,INC_A_51,INC_A_52,...,INC_A_56,INC_A_57,INC_A_60,INC_A_61,INC_A_63,INC_A_65,INC_A_67,INC_A_69,INC_A_71,y
global_id,,,,,,,,,,,,,,,,,,,,,
114801,725134.0,615478.0,23999.0,0.0,6223.0,13900.0,3590.0,0.0,6983.0,0.0,...,127.0,0.0,5155.0,49575.0,0.0,0.0,2845.0,0.0,0.0,-0.314251
114802,525910.0,395442.0,0.0,0.0,0.0,0.0,13800.0,0.0,0.0,0.0,...,0.0,1324.0,19138.0,95502.0,0.0,0.0,0.0,12792.0,0.0,0.012722
114803,1104469.0,923031.0,15400.0,0.0,6518.0,20160.0,9427.0,3293.0,3471.0,0.0,...,3165.0,8132.0,12449.0,97361.0,84.0,1106.0,10808.0,119.0,0.0,-0.441214
114804,2446117.0,1912844.0,303087.0,0.0,25891.0,0.0,29474.0,22570.0,19859.0,0.0,...,23708.0,20735.0,280004.0,-253040.0,3262.0,378308.0,97098.0,8578.0,2805.0,-1.147826
114805,31663846.0,23461410.0,2883640.0,233925.0,113696.0,142144.0,40621.0,279132.0,175023.0,361073.0,...,409085.0,182336.0,1474045.0,854234.0,367.0,439217.0,93836.0,258338.0,159144.0,0.264360
114806,7791150.0,3247837.0,1342832.0,75054.0,79132.0,1530790.0,17299.0,51984.0,54696.0,132987.0,...,35476.0,22098.0,816668.0,288337.0,6.0,12567.0,36301.0,453.0,17066.0,-0.468355
114807,7397089.0,3775244.0,692928.0,67248.0,66927.0,1337827.0,22775.0,9930.0,4262.0,19636.0,...,70343.0,19998.0,770348.0,377504.0,361.0,67879.0,63899.0,168.0,41740.0,-0.254424
114808,1005543.0,627184.0,205661.0,10514.0,7412.0,0.0,5705.0,957.0,4081.0,4600.0,...,4722.0,14383.0,26453.0,93254.0,21197.0,131.0,43342.0,329.0,0.0,0.263592


In [218]:
service.y.describe()

count    7500.000000
mean        0.317133
std        30.467553
min      -759.761905
25%        -0.503774
50%        -0.090626
75%         0.289683
max      1839.645244
Name: y, dtype: float64

y 값 범주화 (상위 25% '1', 그 외 '0')

In [219]:
service['y2']=service['y'] >= 0.289683
service['y2']=service['y2'].astype(float)
service['y2']=service['y2'].astype(str)

In [220]:
service['y3']=service['y'] >= 0
service['y3']=service['y3'].astype(float)
service['y3']=service['y3'].astype(str)

한 것



변수 제거, 결측치 제거+대체, Y생성 ('y'는 본래 수치형 y, 'y2'는 y의 상위 25% 범주형으로 바꿈) 

In [221]:
service.head(8)

,INC_A_41,INC_A_42,INC_A_45,INC_A_46,INC_A_47,INC_A_48,INC_A_49,INC_A_50,INC_A_51,INC_A_52,...,INC_A_60,INC_A_61,INC_A_63,INC_A_65,INC_A_67,INC_A_69,INC_A_71,y,y2,y3
global_id,,,,,,,,,,,,,,,,,,,,,
114801,725134.0,615478.0,23999.0,0.0,6223.0,13900.0,3590.0,0.0,6983.0,0.0,...,5155.0,49575.0,0.0,0.0,2845.0,0.0,0.0,-0.314251,0.0,0.0
114802,525910.0,395442.0,0.0,0.0,0.0,0.0,13800.0,0.0,0.0,0.0,...,19138.0,95502.0,0.0,0.0,0.0,12792.0,0.0,0.012722,0.0,1.0
114803,1104469.0,923031.0,15400.0,0.0,6518.0,20160.0,9427.0,3293.0,3471.0,0.0,...,12449.0,97361.0,84.0,1106.0,10808.0,119.0,0.0,-0.441214,0.0,0.0
114804,2446117.0,1912844.0,303087.0,0.0,25891.0,0.0,29474.0,22570.0,19859.0,0.0,...,280004.0,-253040.0,3262.0,378308.0,97098.0,8578.0,2805.0,-1.147826,0.0,0.0
114805,31663846.0,23461410.0,2883640.0,233925.0,113696.0,142144.0,40621.0,279132.0,175023.0,361073.0,...,1474045.0,854234.0,367.0,439217.0,93836.0,258338.0,159144.0,0.264360,0.0,1.0
114806,7791150.0,3247837.0,1342832.0,75054.0,79132.0,1530790.0,17299.0,51984.0,54696.0,132987.0,...,816668.0,288337.0,6.0,12567.0,36301.0,453.0,17066.0,-0.468355,0.0,0.0
114807,7397089.0,3775244.0,692928.0,67248.0,66927.0,1337827.0,22775.0,9930.0,4262.0,19636.0,...,770348.0,377504.0,361.0,67879.0,63899.0,168.0,41740.0,-0.254424,0.0,0.0
114808,1005543.0,627184.0,205661.0,10514.0,7412.0,0.0,5705.0,957.0,4081.0,4600.0,...,26453.0,93254.0,21197.0,131.0,43342.0,329.0,0.0,0.263592,0.0,1.0


## 예측 변수설명
- y: 영업이익성장률
- y2: 상위 25%기업 여부(1 or 0)
- y3: 성장 여부 (1 or 0)

# test train 나누기

### 기술통계에 잇는것과 없는것으로 train(겹치지않는것) test (겹치는것) 나누기

In [222]:
line3=[]
for i in service.index:
    if i in train.index:
        line3.append(i)
    else:
        line3.append(0)
        
service_test=service[service.index == line3]

service_test['test'] = 1

line4=[]
for i in service.index:
    if not i in train.index:
        line4.append(i)
    else:
        line4.append(0)
        
service_train=service[service.index == line4]

service_train['test'] = 0

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [223]:
col_list = ['INC_A_41', 'INC_A_42', 'INC_A_45', 'INC_A_46', 'INC_A_47', 'INC_A_48',
       'INC_A_49', 'INC_A_50', 'INC_A_51', 'INC_A_52', 'INC_A_55', 'INC_A_56',
       'INC_A_57', 'INC_A_60', 'INC_A_61', 'INC_A_63', 'INC_A_65', 'INC_A_67',
       'INC_A_69', 'INC_A_71']
train_X = service_train[col_list]
train_Y = service_train['y3']
test_X = service_test[col_list]
test_Y = service_test['y3']

## 정규화(로지스팃에서 필수!)

In [224]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#fit_transform 은 fit과 transform을 합친 건데, fit은 일단 각 속성(feature)마다 컬럼을 만드는 작업이라고 생각하면 된다. 이후 transform을 통해 데이터를 변형시키는 거다.
train_features = scaler.fit_transform(train_X)
test_features = scaler.transform(test_X)

# 로지스틱회귀분석
> http://hleecaster.com/ml-logistic-regression-example/
- 여기좋은글 많으니 학습해보기!

In [225]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [226]:
model = LogisticRegression()
model.fit(train_features, train_Y)
print('train 데이터의 스코어: ', accuracy_score(train_Y, model.predict(train_features)))
print('test 데이터의 스코어: ', accuracy_score(test_Y, model.predict(test_features)))
confusion_matrix(test_Y, model.predict(test_features))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


train 데이터의 스코어:  0.5654065406540654
test 데이터의 스코어:  0.5827338129496403


array([[484,   3],
       [345,   2]], dtype=int64)

In [227]:
print('INC_A_41', 'INC_A_42', 'INC_A_45', 'INC_A_46', 'INC_A_47', 'INC_A_48',
       'INC_A_49', 'INC_A_50', 'INC_A_51', 'INC_A_52', 'INC_A_55', 'INC_A_56',
       'INC_A_57', 'INC_A_60', 'INC_A_61', 'INC_A_63', 'INC_A_65', 'INC_A_67',
       'INC_A_69', 'INC_A_71')
print(model.coef_)

INC_A_41 INC_A_42 INC_A_45 INC_A_46 INC_A_47 INC_A_48 INC_A_49 INC_A_50 INC_A_51 INC_A_52 INC_A_55 INC_A_56 INC_A_57 INC_A_60 INC_A_61 INC_A_63 INC_A_65 INC_A_67 INC_A_69 INC_A_71
[[-0.66237944  0.47490292  0.06398973 -0.01796747  0.05467939 -0.01529593
  -0.01940646 -0.00227755 -0.01326272  0.06035583 -0.11045833  0.03649735
  -0.02623278  0.0139909   0.07026184 -0.00337985 -0.004168    0.03984651
  -0.01235652 -0.120221  ]]


# Ridge, Lasso
> https://brunch.co.kr/@itschloe1/11

In [228]:
from sklearn.metrics import r2_score

In [229]:
model = Ridge(alpha = 0.05, normalize =True)
model.fit(train_X, train_Y)
print('train 의 R**2 score:', r2_score(train_Y, model.predict(train_X)))
print('test 의 R**2 score:', r2_score(test_Y, model.predict(test_X)))

train 의 R**2 score: 0.00841315688733335
test 의 R**2 score: -0.002110023848042264


In [230]:
model = Lasso(alpha = 0.05, normalize =True)
model.fit(train_X, train_Y)
print('train 의 R**2 score:', r2_score(train_Y, model.predict(train_X)))
print('test 의 R**2 score:', r2_score(test_Y, model.predict(test_X)))

train 의 R**2 score: 0.0
test 의 R**2 score: -0.0013899197247753126


In [231]:
model = ElasticNet(alpha = 0.05, normalize =True)
model.fit(train_X, train_Y)
print('train 의 R**2 score:', r2_score(train_Y, model.predict(train_X)))
print('test 의 R**2 score:', r2_score(test_Y, model.predict(test_X)))

train 의 R**2 score: 0.0
test 의 R**2 score: -0.0013899197247753126


# GBM
> https://lsjsj92.tistory.com/544?category=853217

In [232]:
model = GradientBoostingClassifier(random_state=0)
model.fit(train_X, train_Y)
print('train 데이터의 스코어: ', accuracy_score(train_Y, model.predict(train_X)))
print('test 데이터의 스코어: ', accuracy_score(test_Y, model.predict(test_X)))
confusion_matrix(test_Y, model.predict(test_X))

train 데이터의 스코어:  0.6834683468346835
test 데이터의 스코어:  0.6211031175059952


array([[404,  83],
       [233, 114]], dtype=int64)

# LGBM
> https://lsjsj92.tistory.com/548

In [233]:
model =LGBMClassifier(n_estimators = 400)
model.fit(train_X, train_Y)
print('train 데이터의 스코어: ', accuracy_score(train_Y, model.predict(train_X)))
print('test 데이터의 스코어: ', accuracy_score(test_Y, model.predict(test_X)))
confusion_matrix(test_Y, model.predict(test_X))

train 데이터의 스코어:  0.9996999699969997
test 데이터의 스코어:  0.5911270983213429


array([[352, 135],
       [206, 141]], dtype=int64)


# XGBOOST
> https://lsjsj92.tistory.com/547
- 여기 위에 사이트 좋으니 여러학습법 학습하면좋을듯

In [236]:
n_estimator = [300]
learning_rates = [0.01, 0.05]
max_depths = [4, 5, 6]
result = []
func_count = 0
for i in n_estimator:
    for j in learning_rates:
        for k in max_depths:
            print(func_count)
            func_count += 1
            model = XGBClassifier(n_estimators = i, learning_rate=j,max_depth=k)
            model.fit(train_X, train_Y)
#             print('{} train 데이터의 스코어: '.format(i), accuracy_score(train_Y, model.predict(train_X)))
            print('estimaors: {}, learning_rate: {}, max_depth: {} Score: '.format(i, j, k), accuracy_score(test_Y, model.predict(test_X)))
#             print(confusion_matrix(test_Y, model.predict(test_X)))
            result.append(accuracy_score(test_Y, model.predict(test_X)))
print(result.index(max(result)), max(result))

0
estimaors: 300, learning_rate: 0.01, max_depth: 4 Score:  0.63189448441247
1
estimaors: 300, learning_rate: 0.01, max_depth: 5 Score:  0.6235011990407674
2
estimaors: 300, learning_rate: 0.01, max_depth: 6 Score:  0.617505995203837
3
estimaors: 300, learning_rate: 0.05, max_depth: 4 Score:  0.6247002398081535
4
estimaors: 300, learning_rate: 0.05, max_depth: 5 Score:  0.6103117505995204
5
estimaors: 300, learning_rate: 0.05, max_depth: 6 Score:  0.5983213429256595
0 0.63189448441247


In [237]:
n_estimator = [300]
learning_rates = [0.01]
max_depths = [4]
result = []
func_count = 0
for i in n_estimator:
    for j in learning_rates:
        for k in max_depths:
            print(func_count)
            func_count += 1
            model = XGBClassifier(n_estimators = i, learning_rate=j,max_depth=k)
            model.fit(train_X, train_Y)
#             print('{} train 데이터의 스코어: '.format(i), accuracy_score(train_Y, model.predict(train_X)))
            print('estimaors: {}, learning_rate: {}, max_depth: {} Score: '.format(i, j, k), accuracy_score(test_Y, model.predict(test_X)))
            print(confusion_matrix(test_Y, model.predict(test_X)))
            result.append(accuracy_score(test_Y, model.predict(test_X)))
print(result.index(max(result)), max(result))

0
estimaors: 300, learning_rate: 0.01, max_depth: 4 Score:  0.63189448441247
[[416  71]
 [236 111]]
0 0.63189448441247


# 랜덤포레스트

In [235]:
model = RandomForestClassifier(n_estimators = 500, max_depth=4)
model.fit(train_X, train_Y)
print('train 데이터의 스코어: ', accuracy_score(train_Y, model.predict(train_X)))
print('test 데이터의 스코어: ', accuracy_score(test_Y, model.predict(test_X)))
confusion_matrix(test_Y, model.predict(test_X))

train 데이터의 스코어:  0.6086108610861086
test 데이터의 스코어:  0.5899280575539568


array([[449,  38],
       [304,  43]], dtype=int64)